<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

# Aller chercher une donnée simple sur Internet

## CSV

Vous trouverez à cette URL un accès aux données de population par pays

<https://population.un.org/wpp/Download/Standard/CSV/>

et plus spécifiquement, le lien pour downloader la donnée

`https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv`

Il s'agit pour nous d'écrire

* un code qui va chercher ces données et les traduit en une structure python (donc dans ce cas précis une simple liste de strings)
* avec une autre URL vous pouvez obtenir toutes les populations de toutes les tranches d'âge de la population brésilienne en 2017
* ou encore calculer combien de petits brésiliens seront nés entre aujourd'hui et demain

## Indices

In [1]:
import json

# il faut installer requests séparément avec
# pip install requests

import requests


In [5]:
# ici j'ai fait un 'Copy Link Address' depuis chrome, l'espace s'est fait remplacer par un %20
# 20 en hexa = 32 en décimal, c'est le codepoint de Espace
URL = "https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv"

# il faut être un peu patient
response = requests.get(URL)


In [6]:
# dans ce genre de cas dir() est utile
# sauf que c'est un peu trop bavard
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [7]:
# mais ça vaut le coup de filter un peu
[symbol for symbol in dir(response) if not symbol.startswith('_')]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [8]:

# si tout s'est bien passé ici on doit avoir 200
# c'est un des codes de retours de HTTP
# si c'est par exemple 404 ça signifie que cette URL n'existe plus
response.status_code

200

In [9]:
# si on était curieux on pourrait aussi faire
response.headers

{'Cache-Control': 'max-age=43200', 'Content-Type': 'application/octet-stream', 'Last-Modified': 'Fri, 06 Mar 2020 20:55:00 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"0ea2a80f9f3d51:0"', 'Server': '', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains, max-age=31536000; includeSubDomains', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Date': 'Wed, 11 May 2022 21:11:53 GMT', 'Content-Length': '22383021'}

In [10]:
# pour voir par exemple la taille de notre donnée
response.headers['Content-Length']

'22383021'

In [11]:
# mais bon, bien sûr ce qu'on veut surtout c'est le contenu

type(response.text), response.text[:100]

(str,
 'LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity\r\n4,Afghanistan,2,M')

In [12]:
# il semble donc qu'on ait affaire à un csv classique
# qu'on peut décortiquer avec object csv.reader
# https://docs.python.org/3/library/csv.html

import csv

# mais pour ça il nous faut un objet de type fichier (file-like object)
# et c'est là que StringIO est super pratique

from io import StringIO

with (StringIO(response.text)) as file_like:
    reader = csv.reader(file_like, delimiter=',')
    # on regarde juste les 3 premiers enregistrements
    for i, row in enumerate(reader, 1):
        print(i, row)
        if i >= 3:
            break


1 ['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'PopMale', 'PopFemale', 'PopTotal', 'PopDensity']
2 ['4', 'Afghanistan', '2', 'Medium', '1950', '1950.5', '4099.243', '3652.874', '7752.117', '11.874']
3 ['4', 'Afghanistan', '2', 'Medium', '1951', '1951.5', '4134.756', '3705.395', '7840.151', '12.009']


À ce stade, on voit comment on peut transformer nos données en listes Python;
mais on va s'arrêter là, parce qu'en pratique ça c'est vraiment un exercice qu'on ferait en pandas...


## JSON

même exercice mais cette fois on cherche du JSON; je tombe en premier sur ceci

<https://github.com/samayo/country-json/blob/master/src/country-by-population.json>

pour trouver le bon lien, je clique sur 'Raw' et je copie ici

In [13]:
URL = "https://raw.githubusercontent.com/samayo/country-json/master/src/country-by-population.json"

In [14]:
def get_url_as_json(url):
    """
    Fetch a URL and decode its result as JSON
    """

    with requests.get(url) as response:
        return json.loads(response.text)

In [15]:
python_friendly = get_url_as_json(URL)

type(python_friendly)

list

In [16]:
python_friendly[:3]

[{'country': 'Afghanistan', 'population': 37172386},
 {'country': 'Albania', 'population': 2866376},
 {'country': 'Algeria', 'population': 42228429}]

In [17]:
# et ici par exemple je pourrais décider que c'est plus pratique
# sous la forme d'un dictionnaire name -> population

population = {d['country']: d['population'] for d in python_friendly}


In [18]:
# de sorte que je peux faire simplement
population['France']

66977107

In [19]:
population

{'Afghanistan': 37172386,
 'Albania': 2866376,
 'Algeria': 42228429,
 'American Samoa': 55465,
 'Andorra': 77006,
 'Angola': 30809762,
 'Anguilla': 15094,
 'Antarctica': 1106,
 'Antigua and Barbuda': 96286,
 'Argentina': 44494502,
 'Armenia': 2951776,
 'Aruba': 105845,
 'Australia': 24982688,
 'Austria': 8840521,
 'Azerbaijan': 9939800,
 'Bahamas': 385640,
 'Bahrain': 1569439,
 'Bangladesh': 161356039,
 'Barbados': 286641,
 'Belarus': 9483499,
 'Belgium': 11433256,
 'Belize': 383071,
 'Benin': 11485048,
 'Bermuda': 63973,
 'Bhutan': 754394,
 'Bolivia': 11353142,
 'Bosnia and Herzegovina': 3323929,
 'Botswana': 2254126,
 'Bouvet Island': 0,
 'Brazil': 209469333,
 'British Indian Ocean Territory': 0,
 'Brunei': 428962,
 'Bulgaria': 7025037,
 'Burkina Faso': 19751535,
 'Burundi': 11175378,
 'Cambodia': 16249798,
 'Cameroon': 25216237,
 'Canada': 37057765,
 'Cape Verde': 543767,
 'Cayman Islands': 64174,
 'Central African Republic': 4666377,
 'Chad': 15477751,
 'Chile': 18729160,
 'China':